In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-15"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7049,2024-07-15,Europa Eurobasket Sub,10:30,Turquia U20,Espanha U20,6.31,1.10,141.5,1.81,1.89,0.0,0.00,0.00,thLifOFF,0.158479,0.909091,0.552486,0.529101,0.067570,95.612,29.766419,0.311325,2.4,1.341641,0.559017,66.96,1.196,0.201569,0.168536,67.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,62,81,1.08,1.02,0.000,0.000,0.994339,0.030578,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7050,2024-07-15,Paraguai Lnb,20:30,Dep. San Jose,Olimpia,1.45,2.61,142.5,1.81,1.87,0.0,0.00,0.00,xtzM4mNF,0.689655,0.383142,0.552486,0.534759,0.072797,130.018,37.413960,0.287760,2.4,1.341641,0.559017,121.66,1.702,0.462839,0.271938,16.0,127.102,38.171222,0.300320,1.2,1.643168,1.369306,162.59,1.724,0.455609,0.264275,6.0,77,71,1.58,2.29,107.010,583.792,0.404061,0.023058,NaN,0.17,0.034,13.235294,0.000000,0.0,0.000000,28.01,5.602,0.287397,0.556177,0.7,0.143823
7051,2024-07-15,Mundo Amistoso Internacional,14:30,Argentina,Sudão do Sul,2.01,1.73,163.5,1.83,1.87,-3.5,1.95,2.09,6gRVqzwH,0.497512,0.578035,0.546448,0.534759,0.075547,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.105877,0.015289,0.049007,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7052,2024-07-15,Mundo Amistoso Internacional,13:00,EUA,Austrália,1.01,9.70,184.5,1.83,1.83,0.0,0.00,0.00,ADGz20Fq,0.990099,0.103093,0.546448,0.546448,0.093192,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.147480,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7053,2024-07-15,Europa Eurobasket Sub,15:30,Montenegro U20,Lituânia U20,9.70,1.01,146.5,1.87,1.83,0.0,0.00,0.00,Yk0Wnt8k,0.103093,0.990099,0.534759,0.546448,0.093192,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,53,62,1.80,1.35,0.000,0.000,1.147480,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7054,2024-07-15,Europa Eurobasket Sub,15:30,Alemanha U20,Israel U20,2.26,1.59,151.5,1.85,1.85,0.0,0.00,0.00,hWvcYtgd,0.442478,0.628931,0.540541,0.540541,0.071409,142.644,62.988203,0.441576,1.2,1.643168,1.369306,100.05,2.038,1.039793,0.510203,-40.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,87,90,1.15,3.77,0.000,0.000,0.246110,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7055,2024-07-15,Europa Eurobasket Sub,13:00,Polónia U20,Sérvia U20,5.67,1.12,151.5,1.85,1.85,0.0,0.00,0.00,bXolz3Oq,0.176367,0.892857,0.540541,0.540541,0.069224,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,75,76,7.50,3.98,0.000,0.000,0.947669,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7056,2024-07-15,Europa Eurobasket Sub,13:00,Macedônia do Norte U20,Bélgica U20,9.70,1.01,133.5,1.80,1.88,0.0,0.00,0.00,lIOah2pS,0.103093

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
